## GraphQL query with Obsidian Notes

I am exposing the notes as a local GraphQL server using this [plugin](https://github.com/TwIStOy/obsidian-local-graphql). 
Access the Apollo client UI by going to this link [http://localhost:28123/](http://localhost:28123/)

In [ ]:
# ! curl --request POST \
#   --header 'content-type: application/json' \
#   --url http://localhost:28123/ \
#   --data '{"query":"query { __typename }"}'

In [1]:
import json
import networkx as nx
from pyvis.network import Network

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [39]:
MIN_NEG_SCORE = -3

def plot_graph(question, result, search_type):
    # Create a graph
    G = nx.Graph()
    G.add_node(question, size=20, title=question)

    # Parse JSON and add nodes and edges
    for item in result["vault"][search_type]:
        if item["score"] > MIN_NEG_SCORE:
            file_name = item["file"]["name"]
            score = item["score"]

            print(file_name)
            print(item['file']['readContent'])
            print("-----------------------------")

            # Adding each file as a node
            G.add_node(file_name, size=10, title=f"Score: {score}")
            
            # Adding an edge from each file to the central query node, weight is the inverse of score for visualization
            G.add_edge(question, file_name, weight=score)

    # Convert to a PyVis network for visualization
    nt = Network("500px", "1000px", notebook=True)
    nt.from_nx(G)
    nt.show("fuzzy_search_graph.html")


In [41]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Configure the client
transport = RequestsHTTPTransport(
    url='http://localhost:28123/',  # Make sure this URL is correct
    use_json=True,
    headers={'Content-Type': 'application/json'},
    verify=True,
    retries=3,
)

question = "LoRA"
search_type = "simpleSearch"  # assuming this is the correct field name you want to replace
client = Client(transport=transport, fetch_schema_from_transport=True)

# Define your query with the variable
string = f""" \
query ExampleQuery($query: String!) {{
  vault {{
    {search_type}(query: $query) {{
      file {{
        name
        readContent
        stat {{
          size
        }}
      }}
      score
    }}
  }}
}}
"""

query = gql(string)
# Variables dictionary
variables = {
    "query": question
}

# Execute the query with variables
result = client.execute(query, variable_values=variables)
# print(result)

# Assuming plot_graph is a function you have defined earlier to create and display the graph
plot_graph(question, result, search_type)


Parameter-Efficient Fine-Tuning (PEFT).md
---
tags:
  - fine-tuning
  - generative-ai
  - llm
---
> [[Large Language Model]], [[Generative AI]], [[Fine Tuning]]

https://github.com/huggingface/peft

**Parameter-Efficient Fine-Tuning (PEFT):**
   - PEFT techniques aim to reduce the computational costs of fine-tuning by limiting parameter updates.
   - One approach, Low-Rank Adaptation (LoRA), posits that not all parameters need updating for effective fine-tuning. By focusing on a low-dimension matrix representative of the task, LoRA can substantially reduce fine-tuning costs.
-----------------------------
Quantization.md
> [[Large Language Model]], [[QLoRA]],
### What is Quantization?

![[Screenshot 2024-03-05 at 13.40.43.png]]

- Quantizing involves splitting a range of numbers (which is infinite in floating point) into bins
- Quantization is needed when you want to represent a value from an infinite range in a physically constrained system, as you need infinite bytes - you need to mak

In [44]:
variables = {
    "query": "QLoRA"
}

# Execute the query with variables
result = client.execute(query, variable_values=variables)
result

{'vault': {'simpleSearch': [{'file': {'name': 'QLoRA.md',
     'readContent': '---\ntags:\n  - fine-tuning\n  - generative-ai\n---\n> [[Fine Tuning]], [[Fine Tuning]], [[Generative AI]], [[Quantization]]\n### QLoRA\n\nThere are 4 components to QLoRA:\n1. **4-bit NormalFloat**\n\t- This is essentially just a way to bucket values\n\t- Representing a value within 4 binary values\n\t- *ex: `0101`*\n\t- 16 Unique unique combinations ![[Screenshot 2024-03-05 at 13.48.24.png]]\n2. **Double Quantization**\n\t- \n3. **Paged Optimizers**\n\t- Used to provide higher utilization of memory across resources\n\t- Instead of being restricted to GPU only memory, you can move pages of memory to the CPU memory, and retrieve when required.\n4. **LoRA**\n\t- Full finetuning would involved reconfiguring weights for entire model - LoRA allows you to  add a small number of trainable parameters to the \n\n![[Screenshot 2024-03-05 at 14.22.14.png]]',
     'stat': {'size': 827}},
    'score': -1.5467},
   {'file